In [14]:
import spacy
import pandas as pd
from spacy import displacy
import os

In [15]:
# Cargar el archivo CSV
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\argumentos_procesados.csv")

# Ver las primeras filas para entender la estructura
print(df.head())

                            titulo  \
0            Argumento de Clitofon   
1               Argumento de Lisis   
2     Argumento de Timeo de Locres   
3               Argumento de Fedro   
4  Argumento de segundo Alcibíades   

                                           argumento  
0  Clitofon, acusado por Sócrates de haber censur...  
1  El objeto de este diálogo es la Amistad, títul...  
2  Timeo, nacido entre los locrios epicefirianos ...  
3  Según una tradición, que no tenemos necesidad ...  
4  No debe orarse ligeramente. Dirigir a los dios...  


In [16]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

In [17]:
# Ejemplo de argumento
ejemplo_argumento = df["argumento"][1]  # Segundo argumento
doc_argumento = nlp(ejemplo_argumento)

In [18]:
# Dependencias sintácticas
displacy.render(doc_argumento, style="dep", jupyter=True)

In [19]:
# Entidades nombradas
displacy.render(doc_argumento, style="ent", jupyter=True)

In [4]:
"""
Funcion procesamiento de texto
Tokenizacion
Lematizacion convertir palabras a su forma base
Eliminacion de stopwords (Quita palabras sin valor semántico)
"""
def procesar_argumento(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

In [5]:
# Procesar el contenido de los argumentos
df["argumento_procesado"] = df["argumento"].apply(procesar_argumento)

# Mostrar resultados procesados
print(df[["titulo", "argumento_procesado"]].head())

                            titulo  \
0            Argumento de Clitofon   
1               Argumento de Lisis   
2     Argumento de Timeo de Locres   
3               Argumento de Fedro   
4  Argumento de segundo Alcibíades   

                                 argumento_procesado  
0  clitofon acusado sócrates censurar conversació...  
1  objeto diálogo amistad título lleno esperanza ...  
2  timeo nacido locrio epicefiriano grecia cuyo n...  
3  tradición necesidad discutir obra juventud pla...  
4  orar él ligeramente dirigir dios súplica pedir...  


In [6]:
"""
Extracción de premisas y conclusiones
Identificamos patrones típicos de premisas ( uso de "porque", "ya que", etc) y conclusiones ( uso de "por lo tanto", "asi que", etc)
"""
def extraer_premisas_conclusiones(texto):
    doc = nlp(texto)
    premisas = [sent.text for sent in doc.sents if any(p in sent.text.lower() for p in ["porque", "ya que", "dado que"])]
    conclusiones = [sent.text for sent in doc.sents if any(c in sent.text.lower() for c in ["por lo tanto", "así que", "en consecuencia"])]
    return {"premisas": premisas, "conclusiones": conclusiones}

In [7]:
df["estructura_argumentativa"] = df["argumento"].apply(extraer_premisas_conclusiones)

# Ejemplo de estructura argumentativa
for index, row in df.iterrows():
    print(f"{row['titulo']} - Estructura Argumentativa:")
    print("Premisas:")
    for premisa in row["estructura_argumentativa"]["premisas"]:
        print(f"{premisa}")
    print("Conclusiones:")
    for conclusion in row["estructura_argumentativa"]["conclusiones"]:
        print(f"{conclusion}")

Argumento de Clitofon - Estructura Argumentativa:
Premisas:
Conclusiones:
Argumento de Lisis - Estructura Argumentativa:
Premisas:
La filosofía quiere más precisión, va al fondo de las cosas; bajo el doble sentido del nombre popular de amigo descubre dos definiciones distintas, que se rechazan entre sí, porque carecen ambas del carácter simple y universal de toda buena definición.
No cabe amistad entre dos hombres, cuyas inclinaciones y afectos no son recíprocos, porque por ambos lados, sin esta reciprocidad, falta algo a la amistad.
No, porque un amigo debe ser útil a su amigo.
En el fondo de todo esto hay una idea muy verdadera, porque estos términos: ni bueno, ni malo, no deben tomarse aquí absolutamente, a la letra, so pena de no designar más que un ser imposible de  determinar, sin carácter ninguno, como sería un hombre sin vicio y sin virtud.
Porque si el mal engendra nuestra amistad por el bien, el bien no tiene existencia sino relativamente al mal, del cual es remedio.
Entonces

In [8]:
"""
Detección de Falacias o Contradicciones
Podemos identificar posibles contradicciones o argumentos débiles basados en la repiticion de ideas opuestas.
"""
def detectar_contradicciones(texto):
    doc = nlp(texto)
    contradicciones = []
    for sent in doc.sents:
        if any(word in sent.text.lower() for word in ["sin embargo", "aunque", "no obstante"]):
            contradicciones.append(sent.text)
    return contradicciones

In [9]:
df["posibles_contradicciones"] = df["argumento"].apply(detectar_contradicciones)

# Mostrar contradicciones encontradas
print(df[["titulo", "posibles_contradicciones"]].head())

                            titulo  \
0            Argumento de Clitofon   
1               Argumento de Lisis   
2     Argumento de Timeo de Locres   
3               Argumento de Fedro   
4  Argumento de segundo Alcibíades   

                            posibles_contradicciones  
0                                                 []  
1  [Sin embargo, el hombre que ama a otro puede n...  
2                                                 []  
3  [,Sin embargo, existe un lazo entre estas dos ...  
4                                                 []  


In [10]:
"""
Filtrado de Argumentos con Estilo Socrático
Identificamos fragmentos que siguen un patron dialéctico o de mayéutica, con preguntas y razonamiento lógico.
"""
def extraer_fragmentos_socraticos(texto):
    doc = nlp(texto)
    fragmentos = [sent.text for sent in doc.sents if "¿" in sent.text or ":" in sent.text]
    return fragmentos

In [12]:
df["fragmentos_socraticos"] = df["argumento"].apply(extraer_fragmentos_socraticos)

# Ejemplo de fragmentos seleccionados
for index, row in df.iterrows():
    print(f"{row['titulo']} - Fragmentos Socráticos:")
    for fragmento in row["fragmentos_socraticos"]:
        print(f"{fragmento}")

Argumento de Clitofon - Fragmentos Socráticos:
Argumento de Lisis - Fragmentos Socráticos:
,¿Es el amigo el que ama o el que es amado?
La segunda definición: que el amigo es aquel que es amado, está expuesta necesariamente a las mismas objeciones.
,De acuerdo con el verso del poeta: , Empedocles ha sostenido que la amistad descansa toda en la semejanza.
En segundo lugar, si la amistad existe entre dos hombre de bien, ¿es la semejanza la que los hace amigos?
¿Qué cosas más contrarias, en efecto, que el odio y la  amistad, lo justo y lo injusto, lo bueno y lo malo?
Y sin embargo, ¿qué cosas menos amigas, o más bien, qué cosas más enemigas?
Prosigue su idea a tientas en cierta manera; le parece que todos los seres deben tener uno de estos tres caracteres: ser buenos, o ser malos, o no ser, ni buenos, ni malos.
Justificada de esta manera la conjetura, se presenta bajo la forma de una definición nueva, a saber: que la amistad consiste en el afecto de lo que no es ni bueno, ni malo, por lo q

In [13]:
# Definir el directorio de salida
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

# Guardar los DataFrames procesados
df.to_csv(os.path.join(output_directory, "argumentos_socraticos_procesados.csv"), index=False)